## YouTube Recomendation system

### Content-based filtering:
### is a recommendation system that makes recommendations to users based on their past interactions with content. The idea behind content-based filtering is to recommend content that is similar to what the user has already shown an interest in.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')

In [ ]:
youtube= pd.read_csv("IN_youtube_trending_data.csv")

In [ ]:
youtube=youtube[:30000]

In [ ]:
youtube.head(2)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,Iot0eF6EoNA,Sadak 2 | Official Trailer | Sanjay | Pooja | ...,2020-08-12T04:31:41Z,UCGqvJPRcv7aVFun-eTsatcA,FoxStarHindi,24,2020-08-12T00:00:00Z,sadak|sadak 2|mahesh bhatt|vishesh films|pooja...,9885899,224925,3979409,350210,https://i.ytimg.com/vi/Iot0eF6EoNA/default.jpg,False,False,Three Streams. Three Stories. One Journey. Sta...
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tan...,2020-08-11T09:00:11Z,UCm9SZAl03Rev9sFwloCdz1g,Rehaan Records,10,2020-08-12T00:00:00Z,[None],11308046,655450,33242,405146,https://i.ytimg.com/vi/x-KbnJ9fvJc/default.jpg,False,False,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...


In [ ]:
youtube.shape

(60000, 16)

In [ ]:
youtube.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

In [ ]:
youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           60000 non-null  object
 1   title              60000 non-null  object
 2   publishedAt        60000 non-null  object
 3   channelId          60000 non-null  object
 4   channelTitle       59999 non-null  object
 5   categoryId         60000 non-null  int64 
 6   trending_date      60000 non-null  object
 7   tags               60000 non-null  object
 8   view_count         60000 non-null  int64 
 9   likes              60000 non-null  int64 
 10  dislikes           60000 non-null  int64 
 11  comment_count      60000 non-null  int64 
 12  thumbnail_link     60000 non-null  object
 13  comments_disabled  60000 non-null  bool  
 14  ratings_disabled   60000 non-null  bool  
 15  description        58815 non-null  object
dtypes: bool(2), int64(5), object(9)
memory u

In [ ]:
YouTube_df = youtube[['video_id','title','channelTitle','categoryId','tags','likes','description','channelId']]

In [ ]:
YouTube_df.head(2)

,video_id,title,channelTitle,categoryId,tags,likes,description,channelId
0,Iot0eF6EoNA,Sadak 2 | Official Trailer | Sanjay | Pooja | ...,FoxStarHindi,24,sadak|sadak 2|mahesh bhatt|vishesh films|pooja...,224925,Three Streams. Three Stories. One Journey. Sta...,UCGqvJPRcv7aVFun-eTsatcA
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tan...,Rehaan Records,10,[None],655450,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...,UCm9SZAl03Rev9sFwloCdz1g


In [ ]:
YouTube_df.title

0        Sadak 2 | Official Trailer | Sanjay | Pooja | ...
1        Kya Baat Aa : Karan Aujla (Official Video) Tan...
2        Diljit Dosanjh: CLASH (Official) Music Video |...
3        Dil Ko Maine Di Kasam Video | Amaal M Ft.Ariji...
4        Baarish (Official Video) Payal Dev,Stebin Ben ...
                               ...                        
59995    Sasural Simar Ka 2 | ससुराल सिमर का | Ep. 50 &...
59996    Taarak Mehta Ka Ooltah Chashmah | तारक मेहता क...
59997    Unboxing Of My New Triumph TIGER 900 RALLY PRO...
59998    ₹1 Lens For My ₹300000 DSLR | क्या एक रूपये का...
59999    Ritik Jash & Mann Collection Vs India’s Riches...
Name: title, Length: 60000, dtype: object

In [ ]:
YouTube_df["title"] = YouTube_df["title"].str.split('|').str[0].str.strip()

In [ ]:
YouTube_df.head(2)

,video_id,title,channelTitle,categoryId,tags,likes,description,channelId
0,Iot0eF6EoNA,Sadak 2,FoxStarHindi,24,sadak|sadak 2|mahesh bhatt|vishesh films|pooja...,224925,Three Streams. Three Stories. One Journey. Sta...,UCGqvJPRcv7aVFun-eTsatcA
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tania,Rehaan Records,10,[None],655450,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...,UCm9SZAl03Rev9sFwloCdz1g


In [ ]:
YouTube_df['tags'] = YouTube_df['tags'].str.replace('|', ' ')

In [ ]:
YouTube_df.head(3)

,video_id,title,channelTitle,categoryId,tags,likes,description,channelId
0,Iot0eF6EoNA,Sadak 2,FoxStarHindi,24,sadak sadak 2 mahesh bhatt vishesh films pooja...,224925,Three Streams. Three Stories. One Journey. Sta...,UCGqvJPRcv7aVFun-eTsatcA
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tania,Rehaan Records,10,[None],655450,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...,UCm9SZAl03Rev9sFwloCdz1g
2,KX06ksuS6Xo,Diljit Dosanjh: CLASH (Official) Music Video,Diljit Dosanjh,10,clash diljit dosanjh diljit dosanjh diljit dos...,296533,CLASH official music video performed by DILJIT...,UCZRdNleCgW-BGUJf-bbjzQg


In [ ]:
YouTube_df.isnull().sum()

video_id           0
title              0
channelTitle       1
categoryId         0
tags               0
likes              0
description     1185
channelId          0
dtype: int64

In [ ]:
YouTube_df.dropna(inplace=True)

In [ ]:
YouTube_df.drop_duplicates(subset=['title'], inplace=True)

In [ ]:
YouTube_df['description'].apply(lambda x:x.split())

0        [Three, Streams., Three, Stories., One, Journe...
1        [Singer/Lyrics:, Karan, Aujla, Feat, Tania, Mu...
2        [CLASH, official, music, video, performed, by,...
3        [Gulshan, Kumar, and, T-Series, presents, Bhus...
4        [VYRL, Originals, brings, to, you, ‘Baarish’, ...
                               ...                        
59983    [#OruchiriIruchiriBumperchiri, #MazhavilManora...
59984    [Click, here, to, watch, the, full, episode, o...
59987    [#madhureddyofficial#madhureddy, emotional, vl...
59989    [https://app.meesho.com/2yoV/d321b785Download,...
59998    [Hello, guys,, in, this, video, We, have, made...
Name: description, Length: 13850, dtype: object

In [ ]:
YouTube_df.head(2)

,video_id,title,channelTitle,categoryId,tags,likes,description,channelId
0,Iot0eF6EoNA,Sadak 2,FoxStarHindi,24,sadak sadak 2 mahesh bhatt vishesh films pooja...,224925,Three Streams. Three Stories. One Journey. Sta...,UCGqvJPRcv7aVFun-eTsatcA
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tania,Rehaan Records,10,[None],655450,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...,UCm9SZAl03Rev9sFwloCdz1g


In [ ]:
YouTube_df['joint_tags'] = YouTube_df['channelTitle'] + YouTube_df['tags'] + YouTube_df['description']

In [ ]:
YouTube_df['joint_tags'][:5]

0    FoxStarHindisadak sadak 2 mahesh bhatt vishesh...
1    Rehaan Records[None]Singer/Lyrics: Karan Aujla...
2    Diljit Dosanjhclash diljit dosanjh diljit dosa...
3    T-Serieshindi songs 2020 hindi songs 2020 new ...
4    VYRLOriginalsVYRL Original Mohsin Khan Shivang...
Name: joint_tags, dtype: object

In [ ]:
YouTube_df.isnull().sum()

video_id        0
title           0
channelTitle    0
categoryId      0
tags            0
likes           0
description     0
channelId       0
joint_tags      0
dtype: int64

In [ ]:
YouTube_df['joint_tags']=YouTube_df['joint_tags'].apply(lambda x:x.split())

In [ ]:
YouTube_df['joint_tags'][1]

['Rehaan',
 'Records[None]Singer/Lyrics:',
 'Karan',
 'Aujla',
 'Feat',
 'Tania',
 'Music/',
 'Desi',
 'Crew',
 'Mix',
 '&',
 'Master',
 '/',
 'Dc',
 'Studio’sVideo/',
 'Sukh',
 'Sanghera',
 'Project',
 'By',
 '/',
 'Deep',
 'Rehaan',
 'Sukh',
 'Bajwa',
 '&',
 'Jeewan',
 'Chahal',
 'Produced',
 'By',
 '/',
 'Sandeep',
 'RehaanLabel',
 '/',
 'Rehaan',
 'RecordsOnline',
 'Promotions',
 '-',
 'Global',
 'Digital',
 'SolutionDigital',
 'Partner',
 '/',
 'Coin',
 'Digital',
 'Social',
 'media',
 'Promotions:',
 'Gk',
 'DigitialSpotify:',
 'https://spoti.fi/3kDfnNuYoutube',
 'Music:',
 'https://bit.ly/31N2tUkGaana:',
 'https://bit.ly/2PHrUkLhttps://wynk.in/u/SSWNC5VO3Website:',
 'WWW.RehaanRecords.CAFB:',
 'https://m.facebook.com/RehaanRecords/INSTA:',
 'Instagram/rehaanrecords']

In [ ]:
new = YouTube_df.drop(columns=['channelTitle','categoryId','tags','likes','description','channelId'])
#new.head()

In [ ]:
new['joint_tags'] = new['joint_tags'].apply(lambda x: " ".join(x))
new.head()

,video_id,title,joint_tags
0,Iot0eF6EoNA,Sadak 2,FoxStarHindisadak sadak 2 mahesh bhatt vishesh...
1,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tania,Rehaan Records[None]Singer/Lyrics: Karan Aujla...
2,KX06ksuS6Xo,Diljit Dosanjh: CLASH (Official) Music Video,Diljit Dosanjhclash diljit dosanjh diljit dosa...
3,UsMRgnTcchY,Dil Ko Maine Di Kasam Video,T-Serieshindi songs 2020 hindi songs 2020 new ...
4,WNSEXJJhKTU,"Baarish (Official Video) Payal Dev,Stebin Ben",VYRLOriginalsVYRL Original Mohsin Khan Shivang...


In [ ]:
new['joint_tags'] = new['joint_tags'].astype('str').str.lower()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,2), max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new['joint_tags']).toarray()

In [ ]:
vector.shape

(13850, 5000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [ ]:
similarity

array([[1.        , 0.0292708 , 0.01113344, ..., 0.01200228, 0.01252577,
        0.02512449],
       [0.0292708 , 1.        , 0.06736033, ..., 0.02297004, 0.01339604,
        0.01394717],
       [0.01113344, 0.06736033, 1.        , ..., 0.02550688, 0.01425185,
        0.00969484],
       ...,
       [0.01200228, 0.02297004, 0.02550688, ..., 1.        , 0.08543605,
        0.01390789],
       [0.01252577, 0.01339604, 0.01425185, ..., 0.08543605, 1.        ,
        0.02391039],
       [0.02512449, 0.01394717, 0.00969484, ..., 0.01390789, 0.02391039,
        1.        ]])

In [ ]:
new[new['title'] == 'Dil Ko Maine Di Kasam Video'].index[0]

3

### Prediction 

In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:10]:
        print(new.iloc[i[0]].title)
        
    

In [ ]:
recommend('Diljit Dosanjh: CLASH (Official) Music Video')

PEED: Diljit Dosanjh (Official) Music Video
Peed Diljit Dosanjh Lyrical Video Song
HABIT Lyric Video: Diljit Dosanjh
Diljit Dosanjh: Whiskey lyrical Video Song
Diljit Dosanjh: Born To Shine (Official Music Video) G.O.A.T
Range Lyric Video
Diljit Dosanjh: Welcome To My Hood (Official Music Video)
@Diljit Dosanjh Welcome To My Hood Song Teaser
Diljit Dosanjh - El Sueno ft. Tru Skool ( Official  Music Video )


In [ ]:
recommend('Baarish (Official Video) Payal Dev,Stebin Ben')

Baarish Ban Jaana (Official Teaser) Payal Dev, Stebin Ben
Baarish Ban Jaana (Official Video) Payal Dev, Stebin Ben
Bepanah Pyaar (Official Teaser) Payal Dev, Yasser Desai
INDIANS and BAARISH
Woh Chaand Kahan Se Laogi (Official Teaser) Vishal Mishra
Barish Ban Jana #Shorts Dance Cover
Bepanah Pyaar (Official Video) Payal Dev, Yasser Desai
Sona Lagda (Official Teaser) Sukriti, Prakriti, Sukhe
Baarish Ki Jaaye


In [ ]:
recommend('Dil Ko Maine Di Kasam Video')

Pachtaoge (Female Version)
Veham Song: Armaan Malik
Chhor Denge: Parampara Tandon
Patli Kamariya
Bewafa Tera Masoom Chehra
Veham Song Teaser: Armaan Malik
TEASER: Pachtaoge (Female Version)
Chhor Denge Teaser ► Parampara Tandon
LUDO: Aabaad Barbaad


In [ ]:
recommend('Naagin 5')

Sasural Simar Ka S2
Molkki
Namak Issk Ka
Ishq Mein Marjawan
Ishq Mein Marjawan S2
Choti Sarrdaarni
Pinjara Khoobsurti Ka
Pinjara Khubsurti Ka
#IdhayathaiThirudathe (Navaratri Special)


In [ ]:
recommend('Fidaa Movie House Tour')

What I Eat In Day
House Warming //home tour// Shiva Jyothi
My Birthday Celebration
My home tour in my village
वट सावित्री व्रत कथा
जेष्ठ पूर्णिमा वट सावित्री व्रत कथा
Bye Bye Baleno
Jyothakka Batukamma Song Promo 2020
A Gift To Our Watchmen Daughter's Wedding


### Some advantages of content-based filtering are that it does not rely on other users' behavior, it can handle new items without a large user base, and it can provide personalized recommendations even for niche interests. However, one limitation is that it may not be able to capture the diversity of user interests, as it tends to recommend similar items to what the user has already consumed.

In [ ]:
import pickle
pickle.dump(new,open('YouTube_list.pkl','wb'))
pickle.dump(similarity,open('YouTube_similarity.pkl','wb'))

## colaborative recomendation ststem

### Scikit-learn's Surprise library is a popular Python library for building collaborative filtering recommendation systems. Here's a general overview of how to build a collaborative recommendation system using the Surprise library:

#### Data preparation: The first step is to prepare your data in a format that can be used by the Surprise library. This typically involves creating a user-item matrix, where each row represents a user and each column represents an item. The values in the matrix represent the user's rating or interaction with the item.

In [ ]:
new = new.reset_index()
titles = new['title']
indices = pd.Series(new.index, index=new['title'])

In [ ]:
indices

title
Sadak 2                                                                                                  0
Kya Baat Aa : Karan Aujla (Official Video) Tania                                                         1
Diljit Dosanjh: CLASH (Official) Music Video                                                             2
Dil Ko Maine Di Kasam Video                                                                              3
Baarish (Official Video) Payal Dev,Stebin Ben                                                            4
                                                                                                     ...  
കോമഡിജോഡീസ് റോക്കിങ് ഓൺ ദി ഫ്ലോർ .!!                                                                 13845
Jethalal को मिला नया Order -Taarak Mehta Ka Ooltah Chashmah -तारक मेहता -Ep 3195 -24th June, 2021    13846
Sudden Surprise akka/చాలా years తరువాత రాజమండ్రీ/పుట్టినీలు Vlog/MadhuReddy official                 13847
I Used Only BLUE Things For 24 

### Create the dataset: Use Surprise's Dataset module to create a dataset object from your Pandas DataFrame.

In [ ]:
reader=Reader()
data = Dataset.load_from_df(youtube[['categoryId','video_id','channelId']], reader)#  'channelId',

### Select an algorithm: Choose a collaborative filtering algorithm to use with the Surprise library. Some popular algorithms include Singular Value Decomposition (SVD), k-Nearest Neighbors (k-NN), and Non-negative Matrix Factorization (NMF).

In [ ]:
from surprise import Dataset, SVD
from surprise.model_selection import GridSearchCV

### Hyperparameter tuning: Finally, you may want to fine-tune the hyperparameters of your model to improve its performance. The Surprise library includes functions for performing grid search or random search to find the optimal hyperparameters for your model.

In [ ]:
svd = SVD()
param_grid = {"n_epochs": [5, 10,20], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

In [ ]:
gs.fit(data)

In [ ]:

# combination of parameters that gave the best RMSE score
print(gs.best_params["mae"])

{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


### Model training & Model evaluation: After training the model, evaluate its performance using metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE)

In [ ]:
svd = SVD( n_epochs= 20, lr_all= 0.005, reg_all= 0.4)
cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=5)

{'test_rmse': array([16.96785267, 16.85351725, 16.92224814, 16.9315509 , 16.86274408]),
 'test_mae': array([15.91768627, 15.75260006, 15.85403571, 15.86445203, 15.77041991]),
 'fit_time': (1.192124605178833,
  1.0768661499023438,
  1.1674988269805908,
  1.3938374519348145,
  1.3192806243896484),
 'test_time': (0.1406707763671875,
  0.15921497344970703,
  0.17833566665649414,
  0.14743852615356445,
  0.2046949863433838)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
#youtube[youtube['likes'] >= 5000]

### Generate recommendations: Use the trained model to generate personalized recommendations for users based on their past interactions with items.

In [ ]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=5, details={'was_impossible': False})

In [ ]:
#youtube[youtube['categoryId'] ==1]

### Hybrid :
### A hybrid recommendation system combines multiple recommendation techniques, such as content-based and collaborative filtering, to provide more accurate and diverse recommendations to users. The system involves collecting user behavior and preference data, selecting and training the recommendation algorithms, combining the recommendations, and evaluating and refining the system. Additional features, such as user demographics, can be incorporated for further personalization. The key is to fine-tune the system to meet the specific needs of the application.

In [ ]:
smd =YouTube_df[['title','likes','categoryId','video_id']]
smd.head(5)

,title,likes,categoryId,video_id
0,Sadak 2,224925,24,Iot0eF6EoNA
1,Kya Baat Aa : Karan Aujla (Official Video) Tania,655450,10,x-KbnJ9fvJc
2,Diljit Dosanjh: CLASH (Official) Music Video,296533,10,KX06ksuS6Xo
3,Dil Ko Maine Di Kasam Video,743931,10,UsMRgnTcchY
4,"Baarish (Official Video) Payal Dev,Stebin Ben",268817,10,WNSEXJJhKTU


In [ ]:
id_map= YouTube_df[['video_id','title','categoryId','channelId']].set_index('title')
id_map.head(2)#	movieId	id  title	

,video_id,categoryId,channelId
title,,,
Sadak 2,Iot0eF6EoNA,24,UCGqvJPRcv7aVFun-eTsatcA
Kya Baat Aa : Karan Aujla (Official Video) Tania,x-KbnJ9fvJc,10,UCm9SZAl03Rev9sFwloCdz1g


In [ ]:
indices_map = id_map.set_index('categoryId')

In [ ]:
indices_map.head(2)

,video_id,channelId
categoryId,,
24,Iot0eF6EoNA,UCGqvJPRcv7aVFun-eTsatcA
10,x-KbnJ9fvJc,UCm9SZAl03Rev9sFwloCdz1g


In [ ]:
id_map.loc['Sadak 2']['categoryId']

24

### Prediction 

In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['video_id']
    #print(idx)
    movie_id =id_map.loc[title]['video_id']
  
    sim_scores = list(enumerate(similarity[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    video_indices = [i[0] for i in sim_scores]
    #print(movie_indices)
    
    video = smd.iloc[video_indices][['title','categoryId','video_id','likes']]
    #movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    video['est'] = video['categoryId'].apply(lambda x: svd.predict(userId, indices_map.iloc[x]['video_id']).est)
   
    video = video.sort_values('likes', ascending=False)
    return video.head(10)

In [ ]:
 svd.predict(1, indices_map.iloc[24]['video_id'])

Prediction(uid=1, iid='lw4ZqLZSBow', r_ui=None, est=5, details={'was_impossible': False})

In [ ]:
hybrid(1,'Types of Ex Girlfriends')

,title,categoryId,video_id,likes,est
43555,Girls In Saloon,22,40XallAwppY,18412,5
58710,Teen vs Kids,22,l1Xd9yhjDVI,17904,5
13001,Desi vs Firangi,22,hrm0pbrIhAg,12583,5
23134,Types of Girls,22,NUQruLO_eK8,12293,5
47538,Types Of Patient,22,f0yt_JTvUP4,11408,5
9622,Couples,22,3Oo0oROejbM,11236,5
36677,Types of Friends,22,BsuyYEVXaC8,11200,5
1704,Indian Maids,22,3szVkjPhs_Y,6788,5
19897,Before Marriage Vs After Marriage,22,XQNNu7EcjTU,6597,5


In [ ]:
hybrid(455,'Sodhi ने उड़ाया Bhide का मज़ाक-Taarak Mehta Ka Ooltah Chashmah-तारक मेहता-Ep 3084 -20th January 2021')

,title,categoryId,video_id,likes,est
28956,कहाँ है Popatlal? - Taarak Mehta Ka Ooltah Cha...,24,wgn4i38Za8M,33063,5
20857,पोपटलाल कहाँ गया? -Taarak Mehta Ka Ooltah Chas...,24,UvxmdXOk6S0,30761,5
17831,Popatlal का एकांत - Taarak Mehta Ka Ooltah Ch...,24,eyEq87OU0qo,23175,5
34629,Jethalal लाया एक Briefcase - Taarak Mehta Ka O...,24,eSxyaDq9JD0,22846,5
29790,Goukuldham हुआ Popatlal से खुश-Taarak Mehta Ka...,24,0hZtPwv_Erw,20882,5
27644,क्यों हुआ Gokuldham परेशान? -Taarak Mehta Ka O...,24,mtAbrgE9vfg,18566,5
31286,Gokuldham ne manaya गणतंत्र दिवस -Taarak Mehta...,24,_w6thZfY8oQ,15595,5
34684,Sodhi ने किया अपने Friends को Ready-Taarak Meh...,24,D1H1jYLjg4k,13918,5
30206,क्या Taarak को मिलेगा Rasgulla?-Taarak Mehta K...,24,jJJy7ghMBUU,11735,5


In [ ]:
hybrid(545,'KGF Chapter2 TEASER Reaction')

,title,categoryId,video_id,likes,est
27368,KGF Chapter2 TEASER,24,Qah9sSIXJqk,4376290,5
27747,KGF Chapter 2 Teaser,24,GiIBNdZTo8Q,487005,5
27378,Sanjay Dutt & Srinidhi Shetty Speaks on KGF Ch...,24,BDK-9U3ZHWs,139455,5
27374,KGF Chapter 2 Teaser Trailer REVIEW,24,LKwirvAB-vk,109334,5
27396,KGF Chapter 2 teaser: akela ayega wo MONSTER 🔥🔥🔥,24,-0SOJVfKX00,34786,5
27781,why is KGF so successful? log itne crazy kyu h...,24,ZrqrIq8tDxk,33370,5
27427,KGF Chapter 2 Teaser Public Talk,24,w7Dn_1ZDizs,14198,5
27816,"Kgf Chapter 2, Budget, Release Date, Box Offic...",24,BeJrCdO8o6g,10345,5
14238,"KGF Chapter 2: KGF 2, Sanjay Dutt, Yash, Prash...",24,lecP8kM4jKs,3289,5


In [ ]:
hybrid(455,'Jethalal Ke Haath Ki Chai')

,title,categoryId,video_id,likes,est
28509,Popatlal Ki Dulhan Ka Swagat!,24,Rl_ud_h4vHw,215031,5
29558,Gokuldham Mein Confusion,24,VIj8vPadc6Y,106503,5
27449,Popatlal Ki Dulhan!,24,RGoyOypjkrY,103679,5
28806,Gokuldham Is Excited To Welcome Popatlal's Bride!,24,tSApbwU7wUw,89750,5
23310,NEW! Ep 3058 - Tapu Sena's Pizza Party!,24,Ev81X0mWdgw,79760,5
27272,Jetha Aur Bhide Ki Fight?!,24,rhCL7mzJXXk,60690,5
47624,NEW! Ep 3152 - Sodhi Ne Bhide Ko Nehlaya!,24,sFEJ-oYCeps,51585,5
47154,NEW! Ep 3151 - Jethalal Visits Bhide's House!,24,qvcgjPdYZ6E,51291,5
23611,NEW! Ep 3059 - Tapu Sena Caught?!,24,YQxUQIBMHHA,50107,5


In [ ]:
hybrid(455,'When 10R meets NINJA H2 !!')

,title,categoryId,video_id,likes,est
33416,Satane Lage Ho (Full Video) Ninja - Ruhi Singh...,10,2WMKa29gRvw,164479,5
59,how JS FILMS bought KAWASAKI H2,22,lh7QPjPpXas,123335,5
3476,UNBOXING SILVER PLAY BUTTON,22,Sdt00Jk7M1c,98218,5
367,Accidental Wheelie on NINJA H2,22,zkArAMNUyAY,53613,5
58776,പണ്ടത്തെ ഈ ഞാൻ 👦 Reacting to my Old Facebook ...,22,rCf3CwS4Z0o,43250,5
10896,why JS Films QUIT YouTube ?,22,l2KT4hbguLQ,29340,5
44786,SURPRISED Everyone with my new SUPERBIKE,22,gcmBri2O5p8,18092,5
14219,Surprising my Friends with NEW Bike!!,22,FG9MXlvkUkc,16773,5
16568,SUBSCRIBERS ne Keh ke LELI 😭😂,22,r9RE9eO_qYs,12130,5


In [ ]:
hybrid(455,'FULL MATCH - Roman Reigns vs. Murphy: SmackDown LIVE, August 13, 2019')

,title,categoryId,video_id,likes,est
14344,Roman Reigns breaks down over brutality to Jey...,17,EcOrKhBh_YY,68318,5
7117,FULL MATCH - Roman Reigns vs. Baron Corbin – U...,17,XsM2Fi8J4zw,48155,5
6971,FULL MATCH - Roman Reigns vs. Drew McIntyre: W...,17,1pb8jp8uOp4,43634,5
13570,FULL MATCH - Shinsuke Nakamura vs. Roman Reign...,17,UG4Wpk5kD0s,40442,5
5469,FULL MATCH: Roman Reigns vs. John Cena: WWE No...,17,hjUsCZ5i1yc,38493,5
5776,FULL MATCH: Roman Reigns vs. Randy Orton vs. K...,17,wk5EeNzH17o,37209,5
2883,FULL MATCH: Roman Reigns vs. Braun Strowman: W...,17,HpqHxoYJ-sQ,35990,5
4625,FULL MATCH: Rusev vs. Roman Reigns – U.S. Titl...,17,1e1-xibMDkU,27412,5
6416,FULL MATCH - Roman Reigns & Braun Strowman vs....,17,cvyG1RV2-28,25577,5


In [ ]:
hybrid(455,'IPL 2020 - Patanjali IPL As Sponsor With 10 Big News')

,title,categoryId,video_id,likes,est
3268,IPL BREAKING: Suresh Raina Pulls Out of IPL Af...,17,dXlrAJ3vJPw,45970,5
7300,IPL 2020 - Match 01,17,lmfk_6ZVoBI,19438,5
4678,"IPL 2020 - Schedule Update,Bhajji Out With 10 ...",17,mZtvQBqMc4Y,15993,5
19229,IPL 2021 - First List Of All 82 Released Playe...,17,qb_QJVXC38o,13401,5
17080,"IPL 2021 New Team Announce, 9 Teams To Play In...",17,hpAtLuncsWk,10345,5
13860,CSK NEW CAPTAIN,24,zwxx85E6Rjo,7413,5
14955,CSK win-ஆல் யாருக்கு லாபம்? யாருக்கு நஷ்டம்? இ...,25,nhQMNGxKMXI,5500,5
48685,KKR v RCB set to be postponed amidst COVID scare,17,Z_JHPYmAWoI,4775,5
7728,IPL 2020 Live : Watch IPL 2020 Live In Mobile ...,17,fE2FQqFBhgs,3517,5


In [ ]:
hybrid(455,"FilterCopy")

,title,categoryId,video_id,likes,est
19631,COVID-19 Vaccine Update,25,kWws30Z6yo4,83691,5
1606,DNA: नौकरियों के लिए अलग-अलग प्रवेश परीक्षाओं ...,25,ceIiCY8MWxY,45259,5
24609,"Coronavirus Mutation से UK में Lockdown लगा, क...",25,Q_incbrBcSc,20194,5
8187,"Film City को लेकर फुल एक्शन मोड में Yogi, बॉली...",25,IPUUafZ4XIw,18130,5
30332,Breaking News: India से संबंधों को लेकर Biden ...,25,M_RHzKpGzuA,14462,5
31142,Taal Thok Ke: Red Fort पर Khalistan का हमला?,25,lzy1PPrGXDU,11993,5
45388,"COVID-19: 24 घंटे बाद होगा Corona पर महामंथन, ...",25,XQxhTh42AMI,10354,5
9550,Hathras Case: पीएम मोदी ने ली हाथरस मामले की ज...,25,mzLyIXJg3Ic,10244,5
40116,আরো একবার Lockdown? Corona -র দ্বিতীয় ঢেউ নিয়ে...,25,b_u0oOvbxJ4,9994,5


In [ ]:
hybrid(455,"DNA: Russia August 12 को लॉन्च करेगा Corona Vaccine")

,title,categoryId,video_id,likes,est
19631,COVID-19 Vaccine Update,25,kWws30Z6yo4,83691,5
1606,DNA: नौकरियों के लिए अलग-अलग प्रवेश परीक्षाओं ...,25,ceIiCY8MWxY,45259,5
24609,"Coronavirus Mutation से UK में Lockdown लगा, क...",25,Q_incbrBcSc,20194,5
8187,"Film City को लेकर फुल एक्शन मोड में Yogi, बॉली...",25,IPUUafZ4XIw,18130,5
30332,Breaking News: India से संबंधों को लेकर Biden ...,25,M_RHzKpGzuA,14462,5
31142,Taal Thok Ke: Red Fort पर Khalistan का हमला?,25,lzy1PPrGXDU,11993,5
45388,"COVID-19: 24 घंटे बाद होगा Corona पर महामंथन, ...",25,XQxhTh42AMI,10354,5
9550,Hathras Case: पीएम मोदी ने ली हाथरस मामले की ज...,25,mzLyIXJg3Ic,10244,5
40116,আরো একবার Lockdown? Corona -র দ্বিতীয় ঢেউ নিয়ে...,25,b_u0oOvbxJ4,9994,5


In [ ]:
hybrid(455,"Yeh Rishta Kya Kehlata Hai: Meet Naira's daughter!")

,title,categoryId,video_id,likes,est
28161,"Ghum Hai Kisikey Pyaar Meiin: 'Slap game' on, ...",24,kxDFEhjFPR0,7200,5
19700,Karwa Chauth celebrated in Kundali Bhagya with...,24,0LT3rzoFGLo,6806,5
6721,Know latest update from the sets of 'Kundali B...,24,07jyiLRhum4,6496,5
27809,Why Virat got angry with Sai?,24,mA5FLbvL8uI,5175,5
15796,Pragya saves Abhi during Dussehra celebrations,24,FexXfg1J7eQ,3757,5
27058,Apna Time Bhi Aayega: Rani Maa stops Veer from...,24,-bHp-v8KUtQ,3001,5
15402,Kundali Bhagya: Karan & Preeta's 'sweet' argument,24,_Ue_Dl5tNMs,2805,5
17485,Kundali Bhagya: Daadi's new drama on Karwa Chauth,24,wukuUB0IcMo,2659,5
40130,Anupamaa gives a HUGE JOLT to Kavya,24,PmqeimC7gr0,2282,5
